In [1]:
import pandas as pd
import os

In [2]:
mdf = pd.read_csv('./mdf.csv')

In [3]:
mdf

,Unnamed: 0,movieId,title,genres
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,2,Jumanji (1995),Adventure|Children|Fantasy
2,2,3,Grumpier Old Men (1995),Comedy|Romance
3,3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...,...
9737,9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,9739,193585,Flint (2017),Drama
9740,9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [4]:
google_api_key = os.environ['GOOGLE_API_KEY']
google_engine_id = os.environ['GOOGLE_ENGINE_ID']

In [5]:
from googleapiclient.discovery import build
import re

In [6]:
def _google_search(search, api_key=google_api_key, cse_id=google_engine_id):
    '''
    Searches a custom search engine for the movie
    '''
    service = build('customsearch', 'v1', developerKey=api_key)
    res = service.cse().list(q=search, cx=cse_id, searchType='image', siteSearch='www.imdb.com').execute()
    return res['items']

def get_movie_poster(search):
    '''
    Gets the IMDb result and returns the movie poster image and IMDb link
    '''
    results = _google_search(search)

    # Filter out brackets for search (The Prestige (2006) --> the prestige)
    search = re.sub(r"[\(\[].*?[\)\]]", "", search)
    search = search.lower()
    for result in results:
        title = result['title'].lower()
        if search in title and ' - imdb' in title:
            return result['link'], result['image']['contextLink']
    return 'No link', 'No contextLink'

In [24]:
res = get_movie_poster('Toy Story')
print(res)

('https://m.media-amazon.com/images/M/MV5BMDU2ZWJlMjktMTRhMy00ZTA5LWEzNDgtYmNmZTEwZTViZWJkXkEyXkFqcGdeQXVyNDQ2OTk4MzI@._V1_.jpg', 'https://www.imdb.com/title/tt0114709/')


In [4]:
# links = pd.DataFrame(columns=['link', 'context'])
links = pd.read_csv('links.csv')
links_copy = pd.read_csv('links_copy.csv')

In [5]:
links

,Unnamed: 0,link,context
0,0,https://m.media-amazon.com/images/M/MV5BMDU2ZW...,https://www.imdb.com/title/tt0114709/
1,1,https://m.media-amazon.com/images/M/MV5BZTk2Zm...,https://www.imdb.com/title/tt0113497/
2,2,https://m.media-amazon.com/images/M/MV5BMjQxM2...,https://www.imdb.com/title/tt0113228/
3,3,https://m.media-amazon.com/images/M/MV5BYzcyMD...,https://www.imdb.com/title/tt0114885/
4,4,https://m.media-amazon.com/images/M/MV5BOTEyNz...,https://www.imdb.com/title/tt0113041/
...,...,...,...
71,71,No link,No contextLink
72,72,No link,No contextLink
73,73,https://m.media-amazon.com/images/M/MV5BMzI0OW...,https://www.imdb.com/title/tt0114660/
74,74,No link,No contextLink


In [6]:
# links_copy = links.copy()
del links_copy['Unnamed: 0']

In [7]:
links_copy

,link
0,https://m.media-amazon.com/images/M/MV5BMDU2ZW...
1,https://m.media-amazon.com/images/M/MV5BZTk2Zm...
2,https://m.media-amazon.com/images/M/MV5BMjQxM2...
3,https://m.media-amazon.com/images/M/MV5BYzcyMD...
4,https://m.media-amazon.com/images/M/MV5BOTEyNz...
...,...
9736,No link
9737,https://upload.wikimedia.org/wikipedia/en/thum...
9738,https://upload.wikimedia.org/wikipedia/en/thum...
9739,https://upload.wikimedia.org/wikipedia/commons...


In [17]:
import requests
import json
import re

def get_wiki_image(search_term):
    search_term = re.sub(r"[\(\[].*?[\)\]]", "", search_term)
    if ', The' in search_term:
        search_term = 'The ' + search_term[:len(search_term) - 6] + ' '
    try:
        WIKI_REQUEST = f'https://en.wikipedia.org/api/rest_v1/page/media-list/{search_term}/0'
        response  = requests.get(WIKI_REQUEST)
        json_data = json.loads(response.text)
        if len(json_data['items']) == 0 or ('srcset' not in json_data['items'][0].keys()):
            try:
                search_term = search_term + '(film)'
                WIKI_REQUEST = f'https://en.wikipedia.org/api/rest_v1/page/media-list/{search_term}/0'
                response  = requests.get(WIKI_REQUEST)
                json_data = json.loads(response.text)
                return 'https:' + json_data['items'][0]['srcset'][0]['src']
            except:
                return 'No link'
        else:
            return 'https:' + json_data['items'][0]['srcset'][0]['src']
    except:
        return 'No link'

In [247]:
for i in range(9000, 9741):
    link = get_wiki_image(mdf['title'][i])
    links_copy.loc[i] = [link]
links_copy

,link
0,https://m.media-amazon.com/images/M/MV5BMDU2ZW...
1,https://m.media-amazon.com/images/M/MV5BZTk2Zm...
2,https://m.media-amazon.com/images/M/MV5BMjQxM2...
3,https://m.media-amazon.com/images/M/MV5BYzcyMD...
4,https://m.media-amazon.com/images/M/MV5BOTEyNz...
...,...
9736,No link
9737,https://upload.wikimedia.org/wikipedia/en/thum...
9738,https://upload.wikimedia.org/wikipedia/en/thum...
9739,https://upload.wikimedia.org/wikipedia/commons...


In [243]:
mdf['title'][8900]

'Inside Out (2015)'

In [46]:
links_copy['link'][511]

'https://upload.wikimedia.org/wikipedia/en/thumb/4/49/Snow_White_1937_poster.png/220px-Snow_White_1937_poster.png'

In [210]:
# for i in range(76, 90):
#     link, context_link = get_movie_poster(mdf['title'][i])
#     links.loc[i] = [link] + [context_link]
# links

In [293]:
links_copy['link'].value_counts()

No link                                                                                                                                              2100
https://upload.wikimedia.org/wikipedia/en/thumb/0/03/The_Up_series_DVD.jpg/250px-The_Up_series_DVD.jpg                                                  6
https://upload.wikimedia.org/wikipedia/commons/thumb/c/ce/Plutchik-wheel.svg/640px-Plutchik-wheel.svg.png                                               6
https://upload.wikimedia.org/wikipedia/commons/thumb/6/6a/Edwin_Booth_Hamlet_1870.jpg/320px-Edwin_Booth_Hamlet_1870.jpg                                 5
https://upload.wikimedia.org/wikipedia/commons/thumb/c/c6/Panamanian_Brothers_%288627444206%29.jpg/640px-Panamanian_Brothers_%288627444206%29.jpg       4
                                                                                                                                                     ... 
https://upload.wikimedia.org/wikipedia/en/thumb/8/8f/The_Last_Dragon_%281985

In [47]:
links_copy.to_csv('links_copy.csv')

In [29]:
movie_list = mdf[mdf['title'].str.contains('(?i)whiplash')]
movie_list

,Unnamed: 0,movieId,title,genres
8466,8466,112552,Whiplash (2014),Drama
9426,9426,166024,Whiplash (2013),(no genres listed)


In [30]:
movie_list.iloc[0]['title']

'Whiplash (2014)'

In [33]:
ind = mdf.loc[mdf['title'] == movie_list.iloc[0]['title']].index[0]

In [34]:
ind

8466